In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# Directories
train_dir = 'data/chest_xray/train'
val_dir = 'data/chest_xray/val'
test_dir = 'data/chest_xray/test'

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [8]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)

Epoch 1/10
163/163 [==============================] - 212s 1s/step - loss: 0.3985 - accuracy: 0.8353 - val_loss: 0.7899 - val_accuracy: 0.6875
Epoch 2/10
163/163 [==============================] - 212s 1s/step - loss: 0.2284 - accuracy: 0.9091 - val_loss: 0.7207 - val_accuracy: 0.6875
Epoch 3/10
163/163 [==============================] - 215s 1s/step - loss: 0.1926 - accuracy: 0.9222 - val_loss: 0.6197 - val_accuracy: 0.6875
Epoch 4/10
163/163 [==============================] - 202s 1s/step - loss: 0.1739 - accuracy: 0.9329 - val_loss: 0.6354 - val_accuracy: 0.8125
Epoch 5/10
163/163 [==============================] - 202s 1s/step - loss: 0.1905 - accuracy: 0.9239 - val_loss: 0.4873 - val_accuracy: 0.8125
Epoch 6/10
163/163 [==============================] - 200s 1s/step - loss: 0.1533 - accuracy: 0.9419 - val_loss: 0.7296 - val_accuracy: 0.7500
Epoch 7/10
163/163 [==============================] - 201s 1s/step - loss: 0.1515 - accuracy: 0.9448 - val_loss: 0.9398 - val_accuracy: 0.6250

In [9]:
# Evaluate on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='binary')

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc:.2f}')


Found 624 images belonging to 2 classes.
20/20 [==============================] - 14s 712ms/step - loss: 0.3935 - accuracy: 0.8798
Test Accuracy: 0.88


In [10]:
# Create model directory in backend/
model_save_path = 'backend/model/pneumonia_model.keras'
model.save(model_save_path)
